In [2]:
import xarray as xr
import xvec
import geopandas as gpd
import pandas as pd
import rioxarray
import rasterio

fn = "F:/EnDI/EnDI_data/snow_data/combine_by_coords.zarr"
ds = xr.open_zarr(fn)
ds_reproj = ds.rio.write_crs("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs +nadgrids=@null +wktext")

fn2= "F:/HMA_glacio_hydro_index/gindex_all_models_scenarioSSP2-45.nc"
ds2=  xr.open_dataset(fn2).xvec.decode_cf()
basins = gpd.GeoDataFrame({"geometry":ds2.geometry}).set_geometry("geometry",crs="EPSG:4326")
basins_reproj = basins.to_crs("+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs +nadgrids=@null +wktext")

In [3]:
ds_reproj = ds_reproj.chunk({'time': 252, 'x':1000, 'y':1000})
ds_reproj

<xarray.Dataset>
Dimensions:        (time: 252, y: 7200, x: 12000)
Coordinates:
  * time           (time) datetime64[ns] 2001-01-01 2001-02-01 ... 2021-12-01
  * x              (x) float64 4.448e+06 4.448e+06 ... 1.001e+07 1.001e+07
  * y              (y) float64 2.224e+06 2.225e+06 ... 5.559e+06 5.56e+06
    crs            int32 0
Data variables:
    snow_fraction  (time, y, x) float32 dask.array<chunksize=(252, 1000, 1000), meta=np.ndarray>

In [4]:
aggregated_mean = ds_reproj.xvec.zonal_stats(
    basins_reproj.geometry, x_coords="x", y_coords="y", stats="mean", method = "iterate"
    )
aggregated_mean

<xarray.Dataset>
Dimensions:        (geometry: 934, time: 252)
Coordinates:
  * time           (time) datetime64[ns] 2001-01-01 2001-02-01 ... 2021-12-01
    crs            int32 0
  * geometry       (geometry) object POLYGON ((7872592.09735436 4485330.40910...
Data variables:
    snow_fraction  (geometry, time) float32 dask.array<chunksize=(1, 252), meta=np.ndarray>
Indexes:
    geometry  GeometryIndex (crs=+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +u ...)

In [5]:
chunked_mean = aggregated_mean.chunk({'time':252, 'geometry':934})
chunked_mean

<xarray.Dataset>
Dimensions:        (geometry: 934, time: 252)
Coordinates:
  * time           (time) datetime64[ns] 2001-01-01 2001-02-01 ... 2021-12-01
    crs            int32 0
  * geometry       (geometry) object POLYGON ((7872592.09735436 4485330.40910...
Data variables:
    snow_fraction  (geometry, time) float32 dask.array<chunksize=(934, 252), meta=np.ndarray>
Indexes:
    geometry  GeometryIndex (crs=+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +u ...)

In [6]:
encoded = chunked_mean.xvec.encode_cf()
encoded

<xarray.Dataset>
Dimensions:             (geometry: 934, part: 947, node: 384743, time: 252)
Coordinates:
    crs                 int32 0
    spatial_ref         int32 0
  * time                (time) datetime64[ns] 2001-01-01 ... 2021-12-01
Dimensions without coordinates: geometry, part, node
Data variables:
    node_count          (geometry) int64 468 178 1133 232 ... 331 873 1175 1392
    part_node_count     (part) int64 468 178 1133 232 750 ... 331 873 1175 1392
    interior_ring       (part) bool False False False ... False False False
    geometry_container  float64 nan
    x                   (node) float64 7.873e+06 7.872e+06 ... 9.716e+06
    y                   (node) float64 4.485e+06 4.486e+06 ... 3.095e+06
    crd_x               (geometry) float64 7.873e+06 7.903e+06 ... 9.716e+06
    crd_y               (geometry) float64 4.485e+06 4.158e+06 ... 3.095e+06
    snow_fraction       (geometry, time) float32 dask.array<chunksize=(934, 252), meta=np.ndarray>

In [7]:
encoded.to_netcdf(path = "F:/EnDI/EnDI_data/snow_data/snow_fraction.nc", mode = "w")

MemoryError: Unable to allocate 2.75 MiB for an array with shape (16, 150, 300) and data type float32